# Event pipeline demo visualization

In [ ]:
# Init
from core.spark_session import generate_spark_session
from core.configuration import parse_configuration
from core.data_objects.bronze.bronze_event_data_object import BronzeEventDataObject
from core.data_objects.silver.silver_event_data_object import SilverEventDataObject

config_path = "/opt/dev/src/tests/test_resources/testing_spark.ini"
spark = generate_spark_session(parse_configuration(config_path))

# Visualize Synthetic Event data

In [ ]:
bronze_events_path = "/opt/dev/sample_data/lakehouse/bronze/mno_events"
bdo = BronzeEventDataObject(spark, bronze_events_path)
bdo.read()
bdo.df.show(10)

In [ ]:
bdo.df.count()

## Visualize Event data

In [ ]:
silver_path = "/opt/data/lakehouse/silver"
silver_events_path = f"{silver_path}/mno_events"
sdo = SilverEventDataObject(spark, silver_events_path)
sdo.read()
sdo.df.show(10)

In [ ]:
sdo.df.count()

## Event Quality checks

In [ ]:
input_path = f"{silver_path}/event_syntactic_quality_metrics_by_column"

df = spark.read.format('parquet').load(input_path)
df.show(20, truncate=False)

In [ ]:
input_path = f"{silver_path}/event_syntactic_quality_metrics_frequency_distribution"

df = spark.read.format('parquet').load(input_path)
df.show()